## Fake News Detection using Deep Learning

In [5]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load and preprocess the dataset
# Assuming `news.csv` has 'text' and 'label' columns, where 'text' is the news content and 'label' is 1 (fake) or 0 (real)
data = pd.read_csv("news_Dataset.csv")


# Fill missing values and convert all entries to strings
X = data['text'].fillna("").astype(str)
y = data['label'].values

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize and pad the text data
vocab_size = 5000
max_len = 200

tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_padded = pad_sequences(X_train_seq, maxlen=max_len)
X_test_padded = pad_sequences(X_test_seq, maxlen=max_len)

# Build the LSTM model
model = Sequential([
    Embedding(vocab_size, 128, input_length=max_len),
    LSTM(64, return_sequences=True),
    Dropout(0.5),
    LSTM(32),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
batch_size = 64
epochs = 5
model.fit(X_train_padded, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)

# Evaluate the model
y_pred = model.predict(X_test_padded)
y_pred = (y_pred > 0.5).astype(int)

accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")



Epoch 1/5


722/722 [==============================] - 269s 365ms/step - loss: 0.2237 - accuracy: 0.9145 - val_loss: 0.1878 - val_accuracy: 0.9400
Epoch 2/5
722/722 [==============================] - 247s 342ms/step - loss: 0.2030 - accuracy: 0.9251 - val_loss: 0.1532 - val_accuracy: 0.9481
Epoch 3/5
722/722 [==============================] - 241s 333ms/step - loss: 0.1109 - accuracy: 0.9630 - val_loss: 0.1257 - val_accuracy: 0.9569
Epoch 4/5
722/722 [==============================] - 231s 320ms/step - loss: 0.0837 - accuracy: 0.9719 - val_loss: 0.1645 - val_accuracy: 0.9510
Epoch 5/5
451/451 [==============================] - 34s 73ms/step
Model Accuracy: 0.95


In [6]:
# Save the trained model
model.save("fake_news_detector_model.h5")


C:\Users\Star\Anaconda3\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [8]:
import tkinter as tk
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Load the saved model
model = load_model("fake_news_detector_model.h5")

# Load the tokenizer (assuming it was saved as tokenizer.pkl using joblib or pickle)
# from joblib import load
# tokenizer = load('tokenizer.pkl')

# Function to preprocess and predict
def predict_news():
    # Get the news text from the entry widget
    news_text = news_entry.get("1.0", tk.END)
    
    # Preprocess the text (tokenize and pad)
    sequence = tokenizer.texts_to_sequences([news_text])
    padded_sequence = pad_sequences(sequence, maxlen=200)
    
    # Predict using the model
    prediction = model.predict(padded_sequence)
    
    # Display the result
    if prediction > 0.5:
        result_label.config(text="Fake News", fg="#ff4d4d")  # Red for Fake News
    else:
        result_label.config(text="Real News", fg="#4CAF50")  # Green for Real News

# Initialize the GUI
root = tk.Tk()
root.title("Fake News Detector")
root.geometry("500x400")
root.configure(bg="#101c4f") 

# Add a heading
heading_label = tk.Label(root, text="Fake News Detection Using Deep Learning", 
                         font=("Helvetica", 18, "bold"), bg="#f2f2f2", fg="#333333")
heading_label.pack(pady=15)

# Add a text box for entering news content
news_entry = tk.Text(root, height=10, width=50, font=("Helvetica", 12), wrap="word", borderwidth=2, relief="groove")
news_entry.pack(pady=10)

# Add a styled button to predict
def on_enter(e):
    predict_button['background'] = '#aaefcf'  
    predict_button['foreground'] = 'Black'

def on_leave(e):
    predict_button['background'] = '#495867'  
    predict_button['foreground'] = 'white'

predict_button = tk.Button(root, text="Check News", command=predict_news, font=("Helvetica", 14, "bold"), 
                           bg="#101c4f", fg="white", padx=20, pady=5, borderwidth=2, relief="raised", cursor="hand2")
predict_button.bind("<Enter>", on_enter)
predict_button.bind("<Leave>", on_leave)
predict_button.pack(pady=15)

# Add a label to show the result
result_label = tk.Label(root, text="", font=("Helvetica", 16, "bold"), bg="#101c4f")
result_label.pack(pady=15)

# Start the GUI loop
root.mainloop()


1/1 [==============================] - 0s 68ms/step


In [ ]:


# Example usage
news_article = """A recent study claims that a new herbal supplement can cure over 90% of chronic illnesses. 
Experts are skeptical, citing the lack of scientific evidence and clinical trials. 
Despite this, the supplement has gained massive popularity on social media."""

